In [1]:
import pandas as pd
import yfinance as yf

news = pd.read_csv('../data/news/noticias_sentimentos.csv')
news['Data'] = pd.to_datetime(news['Data'])
start = news['Data'].min().strftime('%Y-%m-%d')
end = news['Data'].max().strftime('%Y-%m-%d')
print(f'{start} - {end}')

2014-09-08 - 2024-09-08


In [2]:
bvsp = yf.download('^BVSP', start=start, end=end)
bvsp_vol = bvsp['Volume']
bvsp_close = bvsp['Close']
bvsp_returns = bvsp['Close'].pct_change().dropna()

c:\Users\Andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed


In [ ]:
import talib



In [3]:
# BVSP RSI Index
# RSI = 100 - (100 / (1 + RS))
# RS = Average Gain / Average Loss
# Average Gain = Sum of gains over the past 14 periods / 14
# Average Loss = Sum of losses over the past 14 periods / 14

change = bvsp_returns

change_up = change.copy()
change_down = change.copy()

change_up[change_up<0] = 0
change_down[change_down>0] = 0

# Verify that we did not make any mistakes
print(change.equals(change_up+change_down))

# Calculate the rolling average of average up and average down
avg_up = change_up.rolling(14).mean()
avg_down = change_down.rolling(14).mean().abs()

rsi = 100 * avg_up / (avg_up + avg_down)

rsi

True


Date
2014-09-09          NaN
2014-09-10          NaN
2014-09-11          NaN
2014-09-12          NaN
2014-09-15          NaN
                ...    
2024-09-02    62.630231
2024-09-03    55.327626
2024-09-04    59.031232
2024-09-05    61.489806
2024-09-06    44.929015
Name: Close, Length: 2480, dtype: float64

In [4]:
df = pd.DataFrame({'bvsp_vol': bvsp_vol, 'rsi': rsi})
df.to_csv('../data/momentum/ibov_rsi.csv', index=False)